# 3.6 Lab: Linear Regression

## 3.6.1 Libraries

In [ ]:
# in Python, module can be imported by a command similar to 'import numpy as np'. 
# it is a good practice to maintain a section at the beginning of the notebook to import all necessary modules.
# for new module, could use pip to install it. 
# for example 'pip install numpy'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from patsy import dmatrices

In [ ]:
# since in Python, there is no default MASS module and Boston dataset, I will read in the Boston dataset from CSV. The data is in the ./data folder.
Boston = pd.read_csv('data/Boston.csv', header=0)

## 3.6.2 Simple Linear Regression

In [ ]:
# use the commands we learned in the previous chapeter to exame the data.
list(Boston) # or Boston.columns

In [ ]:
Boston.head()

In [ ]:
Boston.shape

In [ ]:
# to have similar formula notation as R, use the following import. 
# One thing to note is that the ' ' for the formula part in fitting step and the '.fit()' at the end.
# import statsmodels.formula.api as smf, we would use smf to call the model. Of course, there are other ways to run linear regression in pythin, such as sklearn.
lm = smf.ols ('medv~lstat', data = Boston).fit()

In [ ]:
print(lm.summary())

In [ ]:
# use dir() to get a list of all the attributes an object has
dir(lm)

In [ ]:
# we can try a few 
print(lm.params)
print(lm.conf_int())

In [ ]:
# provide prediction for 3 observations
lm.predict(pd.DataFrame({'lstat':[5, 10, 15]}))

In [ ]:
# plot the fitted line, we only take two extreme points to make the plot
X_new = pd.DataFrame({'lstat': [Boston.lstat.min(), Boston.lstat.max()]})
preds = lm.predict(X_new)
Boston.plot(kind='scatter', x='lstat', y='medv')
plt.plot(X_new, preds, c='red', linewidth=2)
plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4))= plt.subplots(2, 2)
ax1.plot(Boston.lstat, lm.predict(),'ro')
ax2.plot(lm.predict(), lm.resid, 'go')
ax3.plot(lm.predict(), lm.resid_pearson, 'bo')
plt.show()

In [ ]:
# the statistics of the linear regression mostly stored in lm.get_influence(), for example, the cookdistances, leverage.
dir(lm.get_influence())
# for example, the following identifies the observation with the largest leverage 
np.argmax(lm.get_influence().hat_matrix_diag)

In [ ]:
# from statsmodels.graphics.regressionplots import * just as a reference
plot_leverage_resid2(lm)

In [ ]:
# as mentioned above. For machine learning models, sklearn is the most common used module, but sklearn is a little bit less on statistics.
x = pd.DataFrame(Boston.lstat)
y = Boston.medv
print(x.shape)

model = linear_model.LinearRegression()
model.fit(x, y)
print(model.intercept_)
print(model.coef_)

## 3.6.3 Multiple Linear Regression

In [ ]:
# we can still use smg.ols to run multiple linear regression.
lm = smf.ols ('medv~lstat+age', data = Boston).fit()

In [ ]:
print(lm.summary())

In [ ]:
# if we want to use all the variable. We can use the following trick to manually construct the list. In Python, most of time, you have to manully construct the variable list.
all_columns = "+".join(Boston.columns.difference(["medv"]))
my_formula = "medv~" + all_columns
lm = smf.ols(my_formula, data=Boston).fit()
print(lm.summary())

In [ ]:
# unlike R, Python is not fully up speeded to all the statistics. If you want to have the VIF of the variables in LM, you have to code a little bit.
# from patsy import dmatrices
# from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
y, X = dmatrices(my_formula, data=Boston, return_type='dataframe')
vif_coeff = {}
for i in range(X.shape[1]):
    vif_coeff[X.columns[i]] = variance_inflation_factor(np.array(X.dropna()),i)
    
print(vif_coeff)

## 3.6.4 Interaction Terms

In [ ]:
# we use * to add interaction terms
lm = smf.ols('medv~lstat * age', data=Boston).fit()
print(lm.summary())

## 3.6.5 Non-linear Transformations of the Predictors 

In [ ]:
lm_order1 = smf.ols('medv~ lstat', data=Boston).fit()
lm_order2 = smf.ols('medv~ lstat+ I(lstat ** 2.0)', data=Boston).fit()
print(lm_order2.summary())

In [ ]:
fig, ((ax1, ax2), (ax3, ax4))= plt.subplots(2, 2)
ax1.plot(Boston.lstat, lm_order1.predict(),'ro')
ax3.plot(lm_order1.predict(), lm_order1.resid, 'go')
ax4.plot(lm_order1.predict(), lm_order1.resid_pearson, 'bo')
plt.show()

### if we added in the second order, we can see the residues are more random

In [ ]:
fig, ((ax1, ax2), (ax3, ax4))= plt.subplots(2, 2)
ax1.plot(Boston.lstat,  lm_order2.predict(),'ro')
ax2.plot(Boston.lstat ** 2.0,  lm_order2.predict(),'ro')
ax3.plot(lm_order2.predict(), lm_order2.resid, 'go')
ax4.plot(lm_order2.predict(), lm_order2.resid_pearson, 'bo')
plt.show()

In [ ]:
# there is anova function built in already in statsmodels.  
# if you know what to do, use the key words to google it and likely you will find a very good answer. 
# here we compare the models with one order of stat and two orders of stats. 
# by looking at the p value that will reject the null hypothesis that the coefficent of lstat**2 equals 0.
table = sm.stats.anova_lm(lm_order1, lm_order2)
print(table)

In [ ]:
lm_log = smf.ols('medv~ np.log(rm)', data=Boston).fit()
lm_log.summary()

## 3.6.6 Qualitative Predictors 

I prepared the Carseats file from .Rdata. And it is saved under the data folder.  Let us load them in and explore this dataset.

In [ ]:
Carseats = pd.read_csv('data/Carseats.csv', header=0)

In [ ]:
list(Carseats)

In [ ]:
Carseats.dtypes

In [ ]:
Carseats.head()

In [ ]:
lm_carseats = smf.ols('Sales ~ Income + Advertising + Price + Age', data = Carseats).fit()

In [ ]:
lm_carseats.summary()

In [ ]:
# let us create dummy variables using get_dummies, then exclude the first dummy column
ShelveLoc_dummies = pd.get_dummies(Carseats.ShelveLoc, prefix='ShelveLoc').iloc[:,1:]

In [ ]:
Carseats_dummy = pd.concat([Carseats, ShelveLoc_dummies], axis=1)
Carseats_dummy.head()

In [ ]:
# then the model buliding will be the same with all numerrical variables.
lm_carseats_dummy = smf.ols('Sales ~ Income + Advertising + Price + Age + ShelveLoc_Good + ShelveLoc_Medium', 
                            data = Carseats_dummy).fit()

In [ ]:
# the interpretation of the coefficients are holding everything fixed, Medium shelve location is associated with an average
# increase of sale around 2.0046. 
lm_carseats_dummy.summary() 

In [ ]:
# Compapre the summary of two models, one with explicit encoding of dummy varible, while the other used the built-in function.
lm_carseats_wo_dummy = smf.ols('Sales ~ Income + Advertising + Price + Age + C(ShelveLoc)', 
                            data = Carseats).fit()
lm_carseats_wo_dummy.summary()

## 3.6.7 Writing Functions

In [ ]:
# let us write a simple function to print current time. 
# yhe key word in Python for user defined function is 'def'. 
# pay attention to the ':'. The difference betwwen R (others) and Python is that Python 
# forces you to obey its indentation rules. For example, the following function won't work because of the extra space in front of 'print'.
def print_current_time_wrong():
    from datetime import datetime # this is very bad practice !!! 
    print(str(datetime.now()))  

In [ ]:
def print_current_time():
    from datetime import datetime
    print (str(datetime.now())) 

In [ ]:
print_current_time()

In [ ]:
# End of Chapter 3.